In [1]:
from collections import Counter

from datasets import load_dataset, Dataset
from setfit import SetFitModel, SetFitTrainer
from train.active_learning import ActiveTrainer
from train.train_config import TrainConfig
from train.active_learning_config import ActiveLearningConfig
from data.dataset_config import DatasetConfig

c:\Users\Colin\Documents\aml\aml_final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("sst2")
train_config = TrainConfig(num_iterations=10)
active_learning_config = ActiveLearningConfig()
dataset_config = DatasetConfig()

In [3]:
def model_init():
    return SetFitModel.from_pretrained("thenlper/gte-small", use_differentiable_head=True, head_params={"out_features": dataset_config.num_classes},
)
def after_train_callback(trainer: SetFitTrainer):
    print("Accuraccy: " + str(trainer.evaluate()))
def dataset_callback(dataset: Dataset):
    print("Labels: " + str(Counter(dataset[dataset_config.label_column])))

In [4]:
trainer = ActiveTrainer(
    model_init=model_init, 
    full_train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    train_config=train_config, 
    active_learning_config=active_learning_config, 
    dataset_config=dataset_config,
    after_train_callback=after_train_callback,
    dataset_callback=dataset_callback,
    )

Casting the dataset: 100%|██████████| 8/8 [00:00<?, ? examples/s]


In [5]:
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset
Generating Training Pairs: 100%|██████████| 10/10 [00:00<?, ?it/s]
***** Running training *****
  Num examples = 160
  Num epochs = 1
  Total optimization steps = 20
  Total train batch size = 8
Epoch: 100%|██████████| 1/1 [00:16<00:00, 16.87s/it]
Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.
Epoch: 100%|██████████| 5/5 [01:11<00:00, 14.32s/it]
Applying column mapping to evaluation dataset
***** Running evaluation *****


Accuraccy: {'accuracy': 0.713302752293578}


KeyError: "Column labels not in the dataset. Current columns in the dataset: ['idx', 'sentence', 'label']"